# Healthcare - Data extraction and Summarization in SBAR format

 
## Objective
Clinical/Medical history data extraction and Summarization to structured data in a standard clinical reporting format. Patient data, both current and historical, is usually available in disparate systems and can be put together in a tabular format. However, extracting meaningful insights from this information demands hospital clinicians to navigate through multiple fields, analyze the data, and identify any noteworthy changes over time. By automatically condensing these changes into a standard clinical report format, would not only add substantial value but also serve as a crucial point of differentiation from existing hospital early warning solutions. Utilizing generative AI for this purpose can prove instrumental in uncovering these insights.

## Usecase Details
Extract a wide range of tabular data and summarize it in standard [SBAR](https://en.wikipedia.org/wiki/SBAR) structure. SBAR is an acronym for Situation, Background, Assessment, Recommendation; a technique that can be used to facilitate prompt and appropriate communication. This communication model has gained popularity in healthcare settings, especially amongst professions such as physicians and nurses. It is a way for health care professionals to communicate effectively with one another, and also allows for important information to be transferred accurately. The format of SBAR allows for short, organized and predictable flow of information between professionals. 




## Installation of necessary Library

In [94]:
!python -m pip install amazon-textract-caller --upgrade
!python -m pip install amazon-textract-response-parser --upgrade

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [95]:
!pip install  --force-reinstall boto3

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 231.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 229.4 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 195.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 kB 289.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: jmespath
    Found existing installation: jmespath 1.0.1
    Uninstalling jmespath-1.0.1:
      Successfully uninstalled jmespath-1.0.1
  Attempting uninstall: python-dateutil
    Found existin

## Initial Set up 

In [96]:
import sagemaker
import boto3

import numpy as np                                # For performing matrix operations and numerical processing
import pandas as pd                               # For manipulating tabular data
from time import gmtime, strftime
import os

region = boto3.Session().region_name
smclient = boto3.Session().client('sagemaker')

In [97]:


df = pd.read_csv("data.csv")
#df = df.replace(np.nan, None)

In [98]:
data = df.to_string()
print(data)

     Patient Name         Unnamed: 1               Room       Admitting Dx
0   Sarah Jenkins                NaN             3W-110     mild pneumonia
1            Date    Morning Reading  Afternoon Reading    Evening Reading
2           Day 1          BP:80 bpm          HR:88 bpm          HR:92 bpm
3             NaN    BP: 120/70 mmHg    BP: 118/68 mmHg    BP: 115/65 mmHg
4             NaN         RR: 16 bpm          RR:18 bpm          RR:20 bpm
5             NaN  Rothman Index: 90  Rothman Index: 85  Rothman Index: 80
6           Day 2          HR:95 bpm         HR:100 bpm         HR:105 bpm
7             NaN    BP: 112/60 mmHg        108/58 mmHg    BP: 105/55 mmHg
8             NaN          RR:22 bpm          RR:25 bpm          RR:28 bpm
9             NaN  Rothman Index: 75  Rothman Index: 70  Rothman Index: 65
10          Day 3        HR: 110 bpm         HR:120 bpm         HR:130 bpm
11            NaN    BP: 100/50 mmHg     BP: 95/45 mmHg     BP: 90/40 mmHg
12            NaN        

In [99]:
# Connect to the Bedrock runtime client in us-east-1
bedrock_runtime =boto3.client(service_name='bedrock-runtime',
                                    region_name='us-east-1')

# Zero Shot with Context

In [100]:
prompt_template = """
 \n\nHuman: You are a medical practitioner writing a Medical Summary in standard “SBAR” structure. The below data is csv converted to pandas dataframe \n \n
{input_text}
 \n
 
Write a medical Summary of the above tabular data and summarize the structured data in a standard clinical reporting format. Include medical terminology followed by its meaning. This report is for Hospital clinicians \n \n
Use short bullet point steps \n\n

- Situation: Highlight patient's name and the admitted room. Provide breif summary of the diagnosis, admitting RI scores and changes over the course of the stay
- Background: Highlight patient's reason for admission and brief treatment plan
- Assessment: Summarize the assessment of vital signs over the course of stay
- Recommendation : Provide possible recommendation for any concerning changes in patient's assessment. Highlight which areas doctor should focus on including investigation and monitoring. Also suggest treatment adjustments in the treatment plan.


Assistant:
"""

In [101]:
def format_str(s):
    prompt = prompt_template.replace("{input_text}", s) 
    return prompt

prompt_query=format_str(data)
  
print(prompt_query)


 

Human: You are a medical practitioner writing a Medical Summary in standard “SBAR” structure. The below data is csv converted to pandas dataframe 
 

     Patient Name         Unnamed: 1               Room       Admitting Dx
0   Sarah Jenkins                NaN             3W-110     mild pneumonia
1            Date    Morning Reading  Afternoon Reading    Evening Reading
2           Day 1          BP:80 bpm          HR:88 bpm          HR:92 bpm
3             NaN    BP: 120/70 mmHg    BP: 118/68 mmHg    BP: 115/65 mmHg
4             NaN         RR: 16 bpm          RR:18 bpm          RR:20 bpm
5             NaN  Rothman Index: 90  Rothman Index: 85  Rothman Index: 80
6           Day 2          HR:95 bpm         HR:100 bpm         HR:105 bpm
7             NaN    BP: 112/60 mmHg        108/58 mmHg    BP: 105/55 mmHg
8             NaN          RR:22 bpm          RR:25 bpm          RR:28 bpm
9             NaN  Rothman Index: 75  Rothman Index: 70  Rothman Index: 65
10          Day 3    

In [102]:
import json
body = json.dumps({
                    "prompt": prompt_query,
                    "max_tokens_to_sample":4096,
                    "temperature":0.5,
                    "top_k":250,
                    "top_p":0.5,
                    "stop_sequences": ["\n\nHuman:"]
                  }) 

In [103]:
modelId = 'anthropic.claude-v2' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = bedrock_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print(response_body.get('completion'))

 Here is a draft SBAR medical summary for Sarah Jenkins based on the provided data:

Situation: 
- Patient name: Sarah Jenkins
- Admitted to room 3W-110
- Admitting diagnosis: mild pneumonia
- Admitting Rothman Index: 90 indicating good condition
- Rothman Index declined to 50 over 3 day stay indicating worsening condition

Background: 
- Admitted for treatment of mild pneumonia with antibiotics and monitoring of vital signs.

Assessment:
- Heart rate (HR) increased from 88 bpm on admission to 130 bpm on day 3 indicating tachycardia and possible worsening infection
- Blood pressure (BP) decreased from 120/70 mmHg to 90/40 mmHg indicating hypotension
- Respiratory rate (RR) increased from 16 to 40 breaths/min indicating tachypnea and possible respiratory distress 
- Rothman Index declined from 90 to 50 indicating worsening overall condition

Recommendation:
- Investigate reasons for tachycardia, hypotension and tachypnea including repeat chest imaging to rule out progression of pneumoni

# Chain of Thought - Optional

Optionally you can use different type of prompt engineering technique including chain of thought, many shot prompt engineering

In [104]:
prompt_template_cot = """
 \n\nHuman: You are a medical practitioner writing a Medical Summary in standard “SBAR” structure. The below data is csv converted to pandas dataframe \n \n
{input_text}
 \n
 
Write a medical Summary of the above tabular data and summarize the structured data in a standard clinical reporting format. Include medical terminology followed by its meaning \n \n
Use short bullet point steps \n\n

- Situation: Highlight patient's name and the admitted room. Provide breif summary of the diagnosis, admitting RI scores and changes over the course of the stay
- Background: Highlight patient's reason for admission and brief treatment plan
- Assessment: Summarize the assessment of vital signs over the course of stay
- Recommendation : Provide possible recommendation for any concerning changes in patient's assessment. Highlight which areas doctor should focus on including investigation and monitoring. Also suggest treatment adjustments in the treatment plan.

Think step-by-step for providing a clinical report based on the information shared in the Context before.Include your thoughts in the <thinking></thinking> XML tags and the actual summary as an Answer.



Assistant:
"""

In [105]:
def format_str_cot(s):
    prompt_cot = prompt_template_cot.replace("{input_text}", s) 
    return prompt_cot

prompt_query_cot=format_str_cot(data)
  
print(prompt_query_cot)


 

Human: You are a medical practitioner writing a Medical Summary in standard “SBAR” structure. The below data is csv converted to pandas dataframe 
 

     Patient Name         Unnamed: 1               Room       Admitting Dx
0   Sarah Jenkins                NaN             3W-110     mild pneumonia
1            Date    Morning Reading  Afternoon Reading    Evening Reading
2           Day 1          BP:80 bpm          HR:88 bpm          HR:92 bpm
3             NaN    BP: 120/70 mmHg    BP: 118/68 mmHg    BP: 115/65 mmHg
4             NaN         RR: 16 bpm          RR:18 bpm          RR:20 bpm
5             NaN  Rothman Index: 90  Rothman Index: 85  Rothman Index: 80
6           Day 2          HR:95 bpm         HR:100 bpm         HR:105 bpm
7             NaN    BP: 112/60 mmHg        108/58 mmHg    BP: 105/55 mmHg
8             NaN          RR:22 bpm          RR:25 bpm          RR:28 bpm
9             NaN  Rothman Index: 75  Rothman Index: 70  Rothman Index: 65
10          Day 3    

In [106]:
import json
body_cot = json.dumps({
                    "prompt": prompt_query_cot,
                    "max_tokens_to_sample":4096,
                    "temperature":0.5,
                    "top_k":250,
                    "top_p":0.5,
                    "stop_sequences": ["\n\nHuman:"]
                  }) 

In [107]:
response_cot = bedrock_runtime.invoke_model(body=body_cot, modelId=modelId, accept=accept, contentType=contentType)
response_body_cot = json.loads(response_cot.get('body').read())

print(response_body_cot.get('completion'))

 <thinking>
To write a medical summary in SBAR format for this patient data:

- First, I will highlight the patient's name, admitted room, and diagnosis in the Situation section. I will briefly note the admitting Rothman Index score and changes over the stay.

- Next, for Background I will state the reason for admission and brief treatment plan.

- For Assessment, I will summarize the vital sign trends over the course of the stay, including heart rate, blood pressure, respiratory rate. 

- Finally, in Recommendation I will note any concerning vital sign changes, suggest areas for further investigation and monitoring, and recommend potential treatment plan adjustments based on the assessments.

</thinking>

Here is a draft SBAR medical summary:

Situation: 
- Patient is Sarah Jenkins, a 65 year old female admitted to room 3W-110 with mild pneumonia. Admitting Rothman Index was 90, declining to 50 over course of 3 day stay.

Background: 
- Admitted for intravenous antibiotics to treat co